# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [3]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print(classes)

['Right', 'Crouch', 'Back', 'StandInit', 'Sit', 'Left', 'Belly', 'Stand', 'Knee', 'HeadBack', 'Frog']


In [9]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename, 'rb'))
    target = [i] * len(data)
    return data, target

In [10]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE

for i in range(len(classes)):
    data, target = load_pose_data(i)
    all_data.extend(data)
    all_target.extend(target)
    

print('total number of data', len(all_data))

total number of data 222


In [16]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [17]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [18]:
# YOUR CODE HERE

all_data = np.asarray(all_data)
all_target = np.asarray(all_target)

clf.fit(all_data[training_data], all_target[training_data])

SVC(C=100.0, gamma=0.001)

### predicting

In [19]:
clf.predict(all_data[-1]), all_target[-1]

ValueError: Expected 2D array, got 1D array instead:
array=[-0.40953612 -0.19477606 -1.57997811  2.13682008 -0.40953612  0.28536606
 -1.66136408  2.13997221 -0.03316116  0.98611069].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [21]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [22]:
expected = []
predicted = []
# YOUR CODE HERE

expected = all_target[training_data]
predicted = clf.predict(all_data[training_data])

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      0.83      0.91         6
           1       0.96      1.00      0.98        22
           2       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        35
           4       1.00      1.00      1.00        20
           5       1.00      1.00      1.00        12
           6       1.00      1.00      1.00        13
           7       1.00      1.00      1.00         6
           8       1.00      1.00      1.00         9
           9       1.00      1.00      1.00         8
          10       1.00      1.00      1.00         4

    accuracy                           0.99       155
   macro avg       1.00      0.98      0.99       155
weighted avg       0.99      0.99      0.99       155


Confusion matrix:
[[ 5  1  0  0  0  0  0  0  0  0  0]
 [ 0 22  0  0  0  0  0  0  0  0  0]
 [ 0  0 20  0  0  0  0  0  0  0  0]
 [ 0  0  0 35  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [23]:
expected = []
predicted = []
# YOUR CODE HERE

expected = all_target[permutation[n_training_data:]]
predicted = clf.predict(all_data[permutation[n_training_data:]])

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         8
           2       1.00      1.00      1.00         3
           3       1.00      1.00      1.00        17
           4       1.00      1.00      1.00         6
           5       1.00      1.00      1.00         8
           6       1.00      1.00      1.00         6
           7       1.00      1.00      1.00         5
           8       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         6

    accuracy                           1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00        67


Confusion matrix:
[[ 5  0  0  0  0  0  0  0  0  0  0]
 [ 0  8  0  0  0  0  0  0  0  0  0]
 [ 0  0  3  0  0  0  0  0  0  0  0]
 [ 0  0  0 17  0  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [25]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [29]:
clf2 = pickle.load(open(ROBOT_POSE_CLF, 'rb'))
clf2.predict(all_data[-1:]), all_target[-1:]

(array([10]), array([10]))